In [ ]:
import json
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import norm

warnings.filterwarnings("ignore")

In [ ]:
def read_json_file(json_file):
    with open(json_file) as f:
        data = json.load(f)
    return data


def get_zeros(data):
    zeros = []
    for i in range(len(data["scores"])):
        if data["scores"][i]["label"] == 0:
            zeros.append(data["scores"][i]["score"])
    return zeros


def get_ones(data):
    ones = []
    for i in range(len(data["scores"])):
        if data["scores"][i]["label"] == 1:
            ones.append(data["scores"][i]["score"])
    return ones


def get_labels(data):
    labels = []
    for i in range(len(data["scores"])):
        labels.append(data["scores"][i]["label"])
    return labels

In [ ]:
def plot_scores(stats_file):
    data = read_json_file(stats_file)
    zeros = get_zeros(data)
    ones = get_ones(data)
    title = stats_file.split(".")[0]

    mean_zeros = np.mean(zeros)
    mean_ones = np.mean(ones)
    std_zeros = np.std(zeros)
    std_ones = np.std(ones)

    step = len(zeros)
    x = np.linspace(0, 1, step)

    pdf_zeros = norm.pdf(x, mean_zeros, std_zeros)
    pdf_ones = norm.pdf(x, mean_ones, std_ones)
    idx = np.argwhere(np.diff(np.sign(pdf_zeros - pdf_ones))).flatten()

    sns.set_context("paper")
    sns.set_palette("Set2")

    sns.distplot(zeros, hist=False, kde=True, kde_kws={"shade": True, "linewidth": 2})
    sns.distplot(ones, hist=False, kde=True, kde_kws={"shade": True, "linewidth": 2})

    plt.axvline(x=x[idx], color="black", linestyle="--", linewidth=1)
    plt.text(
        x[idx] + 0.05,
        1,
        f"th_eer={x[idx][0]:.3f}",
        rotation=90,
        fontsize=10,
    )

    plt.legend(["Spoof", "Real"], loc="upper right", fontsize=10)

    plt.title(f"LCC, eer={data['m_eer']:.3f}", fontsize=10, loc="left")

    plt.show()

In [ ]:
stats_dir = "../logs/siwm2lcc/stats/siwm2lcc-test/epoch_000.json"
plot_scores(stats_dir)

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import norm


def plot_scores_dir(stats_dir):
    folders = [
        "makeup_cosmetic",
        "makeup_impersonation",
        "makeup_obfuscation",
        "mannequin",
        "mask_halfmask",
        "mask_papermask",
        "mask_transparentmask",
        "paper",
        "partial_eye",
        "partial_funnyeyeglasses",
        "partial_mouth",
        "partial_paperglasses",
        "replay",
        "silicone",
    ]

    num_cols = 5
    num_rows = 3
    num_plots = num_cols * num_rows

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 9))

    for i, folder in enumerate(folders):
        if i >= num_plots:
            break

        filename = os.path.join(stats_dir, folder, "epoch_009.json")
        data = read_json_file(filename)  # You'll need to implement the read_json_file function
        zeros = get_zeros(data)  # You'll need to implement the get_zeros function
        ones = get_ones(data)  # You'll need to implement the get_ones function

        sns.set_context("paper")
        sns.set_palette("Set2")

        ax = axes[i // num_cols, i % num_cols]  # Calculate the correct position of the subplot

        sns.distplot(zeros, hist=False, kde=True, kde_kws={"shade": True, "linewidth": 2}, ax=ax)
        sns.distplot(ones, hist=False, kde=True, kde_kws={"shade": True, "linewidth": 2}, ax=ax)

        mean_zeros = np.mean(zeros)
        mean_ones = np.mean(ones)
        std_zeros = np.std(zeros)
        std_ones = np.std(ones)

        step = len(zeros)
        x = np.linspace(0, 1, step)

        pdf_zeros = norm.pdf(x, mean_zeros, std_zeros)
        pdf_ones = norm.pdf(x, mean_ones, std_ones)
        idx = np.argwhere(np.diff(np.sign(pdf_zeros - pdf_ones))).flatten()

        ax.axvline(x=x[idx], color="black", linestyle="--", linewidth=1)
        ax.text(
            x[idx] + 0.01,
            2,
            f"th_eer={x[idx][0]:.3f}",
            rotation=90,
            fontsize=10,
        )

        ax.legend(["Spoof", "Real"], loc="upper right", fontsize=10)
        ax.set_title(f"{folder}, eer={data['m_eer']:.3f}", fontsize=10)

    for i in range(len(folders), num_plots):
        axes.flatten()[i].axis("off")  # Hide the remaining empty subplots

    plt.tight_layout()
    plt.show()

In [ ]:
scores_dir = "../logs/siwm-loo/stats"
plot_scores_dir(scores_dir)